In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\vinay\OneDrive\Desktop\DataScienceCleanStatement.csv")

In [3]:
df.head()

,Date,Description,InternationTransactionAmount,Category,Amount,TransactionUserName,TransactionType,IsoCurrency,Locality,City,State,Country,Unnamed: 12
0,12/12/2019,THE ROOM GURGAON ...,0,Video game,-127,VIBHORE GOYAL,Domestic,INR,Central Park II,Gurugram,Haryana,India,NaN
1,12/12/2019,CUT AND STYLEGURGAON ...,0,Occupation,-499,JASJIT KAUR,Domestic,INR,Sector 4,Gurugram,Haryana,India,NaN
2,12/12/2019,MUMUSOGURGAON ...,0,,-388,JASJIT KAUR,Domestic,INR,Sector 49,Gurugram,Haryana,India,NaN
3,12/12/2019,THE ROOM GURGAON ...,0,Video game,-108,VIBHORE GOYAL,Domestic,INR,Central Park II,Gurugram,Haryana,India,NaN
4,12/12/2019,THE ROOM GURGAON ...,0,Video game,-50,VIBHORE GOYAL,Domestic,INR,Central Park II,Gurugram,Haryana,India,NaN


In [4]:
df.isnull().sum()

Date                                                                      0
Description                                                               0
InternationTransactionAmount                                              0
  Category                                                                0
  Amount                                                                  0
TransactionUserName                                                       0
 TransactionType                                                          0
IsoCurrency                                                               0
Locality                                                                168
City                                                                    200
State                                                                   225
Country                                                                 163
Unnamed: 12                                                            1466
dtype: int64

In [5]:
def replace_empty_spaces(df):
    df.replace(r'^\s*$', np.nan, regex=True,inplace=True)
    df.drop(columns=["Unnamed: 12"],inplace=True)

In [6]:
def order_column_name(df):
    columns = []
    for col in df.columns:
        columns.append(col.strip())
    return columns

In [7]:
def change_dtype_money1(df):
    df = df[pd.to_numeric(df['InternationTransactionAmount'], errors='coerce').notnull()]
    return df['InternationTransactionAmount'].astype(float)

In [8]:
def change_dtype_money2(df):
    df = df[pd.to_numeric(df['Amount'], errors='coerce').notnull()]
    return df['Amount'].astype(float)

In [9]:
def correct_date_format(date):
    return date.replace('-','/')

In [10]:
def change_neg_value(df):
    return df['Amount'].apply(lambda x:abs(x))

In [11]:
def handle_missing(df):
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            if df[col].dtype == np.dtype('object'):
                df[col] = df[col].replace(np.nan,df[col].mode()[0])
            elif df[col].dtype == np.dtype('float64'):
                df[col] = df[col].replace(np.nan,df[col].mean())

In [12]:
def strip_string_values(df):
    for col in df.columns:
        if df[col].dtype == np.dtype('object'):
            df[col] = df[col].apply(lambda x:x.strip())

In [13]:
def categorize(s):
    category = ''
    if any(x in s for x in ('Cafe', 'Restaurent', 'Food','food','cafe','restaurent','Coffee','Dish','Pasta','Cocktail','Algae','Supermarket')):
        category = 'Food'
    elif any(x in s for x in ('Hotel','Occupation','Massif','Railway','City','California','Hospitality','Italian','Airline','airline','Agency','US','Oceanic','Palace','Jaipur','Continent','minister','Political','Ridesharing','President','Governer','Ethno-linguistic','borough','director')):
        category = 'Travel'
    elif any(x in s for x in ('Video','game','Cinema','sitcom','Event','Novel','Book','Fictional','Newspaper','television','actress','Poem','actor','film','Song','study','Broadcast','model','Entertainment','Baseball','Television','Radio','Album','singer','song','festivity','Band','art','football','VFX')): ##Enterntainment
        category = 'Entertainment'
    elif any(x in s for x in ('Retail','Clothing','Department','Fashion','Car','Cosmetics','Mobile phone','marketing')): ##Shopping
        category = 'Shopping'
    elif any(x in s for x in ('Animal','Dog')): ##Pet
        category = 'Pets'
    elif any(x in s for x in ('Medication','Operation','chemical','Health')): ##Health
        category = 'Health'
    elif any(x in s for x in ('commerce','corporation','Telecommunications','Electronics','Technology','Hardware','Broadcasting','Mobile network')): ##Utilities
        category = 'Utilities'
    elif any(x in s for x in ('Insurance')): ##Insurence
        category = 'Insurance'
    else:
        category = 'Other'
    return category

In [14]:
def pre_process(data):
    df = data
    replace_empty_spaces(df)
    df.columns = order_column_name(df)
    df['Date'] = df.apply(lambda x:correct_date_format(x['Date']),axis=1)
    df['Date'] = pd.to_datetime(df['Date'],format='%d/%m/%Y')
    df['InternationTransactionAmount'] = change_dtype_money1(df)
    df['Amount'] = change_dtype_money2(df)
    df['Amount'] = change_neg_value(df)
    handle_missing(df)
    strip_string_values(df)
    df['MainCategory'] = df.apply(lambda x:categorize(x['Category']),axis=1)
    return df

In [15]:
df = pre_process(df)

In [16]:
df.head()

,Date,Description,InternationTransactionAmount,Category,Amount,TransactionUserName,TransactionType,IsoCurrency,Locality,City,State,Country,MainCategory
0,2019-12-12,THE ROOM GURGAON,0.0,Video game,127.0,VIBHORE GOYAL,Domestic,INR,Central Park II,Gurugram,Haryana,India,Entertainment
1,2019-12-12,CUT AND STYLEGURGAON,0.0,Occupation,499.0,JASJIT KAUR,Domestic,INR,Sector 4,Gurugram,Haryana,India,Travel
2,2019-12-12,MUMUSOGURGAON,0.0,Cafe company,388.0,JASJIT KAUR,Domestic,INR,Sector 49,Gurugram,Haryana,India,Food
3,2019-12-12,THE ROOM GURGAON,0.0,Video game,108.0,VIBHORE GOYAL,Domestic,INR,Central Park II,Gurugram,Haryana,India,Entertainment
4,2019-12-12,THE ROOM GURGAON,0.0,Video game,50.0,VIBHORE GOYAL,Domestic,INR,Central Park II,Gurugram,Haryana,India,Entertainment


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Date                          1470 non-null   datetime64[ns]
 1   Description                   1470 non-null   object        
 2   InternationTransactionAmount  1470 non-null   float64       
 3   Category                      1470 non-null   object        
 4   Amount                        1470 non-null   float64       
 5   TransactionUserName           1470 non-null   object        
 6   TransactionType               1470 non-null   object        
 7   IsoCurrency                   1470 non-null   object        
 8   Locality                      1470 non-null   object        
 9   City                          1470 non-null   object        
 10  State                         1470 non-null   object        
 11  Country                       

In [18]:
df.to_csv('processed_data.csv',index=False)

In [72]:
username = 'VIBHORE GOYAL'
current_date = '2019-05-12'

<b>Implementation: </b><br>
1) First I selected the data satisfying the conditions i.e "Username" and "Category" and "CurrentDate"<br>
2) Then I used DateOffset method to calculate last month by using current month.<br>
3) Then I used pandas's grouper method to make groups of months using the "Date" column<br>
4) Then I used pandas's groupby method to group the data by months and calculate the sum of "Amount"<br>
5) Voila! The food expenditure of a particular person in a given date range of previous month is calculated.<br>

In [73]:
def check_expenditure(username,category,current_date,df):
    current_date = pd.to_datetime(current_date)
    last_month = current_date - pd.DateOffset(months=1)
    amount = df[(df['TransactionUserName']==username) & (df['MainCategory']==category) & (df['Date'].dt.month==last_month.month)].groupby(pd.Grouper(freq='M', key='Date'))['Amount'].sum()[0]
    print("₹",int(amount))

In [74]:
check_expenditure(username,'Food',current_date,df)

₹ 204485


<b>Implementation: </b><br>
1) First I selected the data satisfying the conditions i.e "Username" and "Category" and "CurrentDate"<br>
2) Then I used DateOffset method to calculate last 3 months by using current month.<br>
3) Then I used pandas's grouper method to make groups of months using the "Date" column<br>
4) Then I used pandas's groupby method to group the data by months and calculate the sum of "Amount"<br>
5) Voila! The travel expenditure of a particular person in a given date range of last 3 months is calculated.<br>

In [82]:
def check_expenditure_last3m(username,category,current_date,df):
    current_date = pd.to_datetime(current_date)
    last3mon = current_date - pd.DateOffset(months=3)
    print(df[(df['TransactionUserName']==username) & (df['MainCategory']==category) & (df['Date']>last3mon) & (df['Date']<current_date)].groupby(pd.Grouper(freq='M', key='Date'))['Amount'].sum()[:3])

In [83]:
check_expenditure_last3m(username,'Travel',current_date,df)

Date
2019-02-28    64916.74
2019-03-31    15590.00
2019-04-30     4993.59
Freq: M, Name: Amount, dtype: float64


<b>Implementation: </b><br>
1) First I selected the data satisfying the conditions i.e "Username" and "Category" i.e Shopping<br>
2) Then I used pandas's grouper method to make groups of months using the "Date" column<br>
3) Then I used pandas's groupby method to group the data by months and calculate the average of "Amount"<br>
4) Then I checked if monthly average Shopping expenditure is greater than 5000 or not.<br>
5) If it was above 5000 then I categorized that person as "Impulsive Shopper"

In [92]:
def is_impulsive(username):
    if df[(df['TransactionUserName']==username)&(df['MainCategory']=='Shopping')].groupby(pd.Grouper(freq='M', key='Date'))['Amount'].sum().mean() > 5000:
        print("IMPULSIVE SHOPPER")
    else:
        print("COMPULSIVE SHOPPER")

In [93]:
is_impulsive(username)

COMPULSIVE SHOPPER
